In [1]:
import tensorflow as tf
from flask import Flask, render_template, request,url_for,redirect  
from werkzeug.utils import secure_filename
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten



In [ ]:
import time
app = Flask(__name__)

@app.route('/')
def main():
    return render_template('index.html')

@app.route('/uploader', methods = ['GET', 'POST'])
def upload_file():
    f = request.files['pic']
    print(f.filename)
    f.save(secure_filename(f.filename))
    model = Sequential()
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
    model.compile(optimizer='rmsprop',loss='mse')
    
    #model = load_model('Grey2colormodel.h5')
    
    path = secure_filename(f.filename)
    image = img_to_array(load_img(path, target_size=(400, 400)))
    image = np.array(image, dtype=float)
    X = rgb2lab(1.0/255*image)[:,:,0]
    Y = rgb2lab(1.0/255*image)[:,:,1:]
    Y /= 128
    X = X.reshape(1, 400, 400, 1)
    Y = Y.reshape(1, 400, 400, 2)
    model.fit(x=X, y=Y,batch_size=1,epochs=0)
    model.load_weights('Grey2colormodel.h5')
    print(model.evaluate(X, Y, batch_size=1))
    output = model.predict(X)
    output *= 128
    # Output colorizations
    cur = np.zeros((400, 400, 3))
    cur[:,:,0] = X[0][:,:,0]
    cur[:,:,1:] = output[0]
    base="static//"
    out_result_path="img_result"+str(time.time())+".png"
    out_grey_path="img_gray_version"+str(time.time())+".png"
    imsave(base+out_result_path, lab2rgb(cur))
    imsave(base+out_grey_path, rgb2gray(lab2rgb(cur)))
    return render_template('output.html',p1=base+out_grey_path,p2=base+out_result_path)
    
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
!pwd